In [25]:
# Biopython 提供了 PDB 模块，可以方便地解析 PDB 并计算原子距离。
from Bio.PDB import PDBParser
from Bio.PDB.Polypeptide import is_aa

# 解析 PDB 文件
pdb_file = "protein.pdb"
parser = PDBParser(QUIET=True)
structure = parser.get_structure("protein", pdb_file)

# 定义残基编号和链
resi1, resi2 = 45, 47
chain_id = "D"

# 提取 Cα 原子
def get_ca_atom(structure, chain_id, resi):
    for model in structure:
        chain = model[chain_id]
        for res in chain:
            if is_aa(res) and res.get_id()[1] == resi:
                return res['CA']
    return None

ca1 = get_ca_atom(structure, chain_id, resi1)
ca2 = get_ca_atom(structure, chain_id, resi2)

# 计算距离
dist = ca1 - ca2  # Biopython 重载了 '-' 运算符
print(f"残基 {resi1} 和 {resi2} 的 Cα 距离为 {dist:.2f} Å")

残基 45 和 47 的 Cα 距离为 5.79 Å


In [26]:
# 使用 Python 文本文件操作
import math

def parse_pdb_ca(filename, resi_list, chain_id=None):
    """
    从 PDB 文件中获取指定残基的 CA 坐标
    :param filename: PDB 文件路径
    :param resi_list: 需要提取的残基编号列表，如 [45, 47]
    :param chain_id: 链 ID，可选
    :return: dict, {resi: (x, y, z)}
    """
    coords = {}
    with open(filename) as f:
        for line in f:
            if line.startswith("ATOM") and line[12:16].strip() == "CA":
                resi = int(line[22:26].strip())
                chain = line[21].strip()
                if chain_id and chain != chain_id:
                    continue
                if resi in resi_list:
                    x = float(line[30:38].strip())
                    y = float(line[38:46].strip())
                    z = float(line[46:54].strip())
                    coords[resi] = (x, y, z)
    return coords

def distance(coord1, coord2):
    """计算两点间欧氏距离"""
    return math.sqrt(sum((a-b)**2 for a, b in zip(coord1, coord2)))

# 使用示例
pdb_file = "protein.pdb"
resi1, resi2 = 45, 47
chain_id = "D"
coords = parse_pdb_ca(pdb_file, [resi1, resi2], chain_id=chain_id)
dist = distance(coords[resi1], coords[resi2])
print(f"残基 {resi1} 和 {resi2} 的 Cα 距离为 {dist:.2f} Å")

残基 45 和 47 的 Cα 距离为 5.79 Å


In [ ]:
import pandas as pd
import numpy as np
df = pd.read_csv('experiment_value.csv')
resname_value_dict = df[['rename','value']].set_index('rename')['value'].to_dict()
filename = "protein.pdb"
new_lines = []
with open(filename) as f:
    for line in f:
        if line.startswith("ATOM"):
            resi = int(line[22:26].strip())
            # 把resname=x的b-factor改掉。
        else:
            new_lines.append(line)
with open('protein_bfactor.pdb') as f_out:
    for line in new_lines:
        f_out.write(line)

In [22]:
# 运行dssp计算蛋白质的二级结构（α-helix, β-sheet 等），除了需要安装biopython的包，还需要安装dssp,他其实调用外部命令行工具 mkdssp 来做计算的。
# conda install -c salilab dssp

from Bio.PDB import PDBParser, DSSP

pdb_file = "protein.pdb"
parser = PDBParser(QUIET=True)
structure = parser.get_structure("protein", pdb_file)

model = structure[0]  # 第一模型
dssp = DSSP(model, pdb_file)

for key in list(dssp.keys())[:5]:
    res_id = key[1][1]
    ss = dssp[key][2]  # 二级结构类型
    print(f"残基 {res_id} 二级结构: {ss}")

残基 20 二级结构: -
残基 21 二级结构: E
残基 22 二级结构: E
残基 23 二级结构: E
残基 24 二级结构: E


In [ ]:
# 可以统计整个结构的螺旋/片层/coil 比例：

from collections import Counter

ss_list = []
for key in list(dssp.keys())[:5]:
    ss_list.append(dssp[key][2])
print(ss_list)
ss_counts = Counter(ss_list)
total = sum(ss_counts.values())

for k, v in ss_counts.items():
    print(f"{k or ' '} : {v/total:.2%}")

['-', 'E', 'E', 'E', 'E']
- : 20.00%
E : 80.00%
